 <!-- test -->


CV(컴퓨터 비전) 프로젝트의 파이프라인을 구체적인 단계와 실행 내용 정리

1. 데이터 준비 및 전처리
데이터 로드: Kaggle Fashion-MNIST 데이터셋의 훈련용(60,000개)과 테스트용(10,000개) 이미지를 PyTorch 라이브러리를 사용해 불러왔습니다.
정규화: 모든 28x28 픽셀의 이미지 데이터를 0~255 범위의 정수에서 0~1 범위의 부동 소수점 값으로 변환했습니다. 이 과정은 모델이 학습하기 쉽도록 데이터를 표준화하는 데 필수적입니다.
2. 모델 구축
모델 아키텍처 정의: torch.nn.Module 클래스를 상속받아 FashionMNISTModel 클래스를 직접 정의했습니다.
레이어 설계: 이미지를 1차원 벡터로 변환하는 Flatten 레이어와 두 개의 Linear (완전 연결) 레이어를 순서대로 쌓아 신경망을 구성했습니다.
모델 개선: 초기 모델의 성능을 향상시키기 위해, DeeperFashionMNISTModel을 정의하여 레이어를 3개로 늘리고 뉴런 수를 증가시켜 모델의 표현력을 높였습니다.
3. 모델 학습
장치 설정: 학습을 빠르게 진행하기 위해 Apple Silicon의 GPU를 사용하는 mps 장치를 설정했습니다.
학습 환경 구성: 모델의 오차를 계산하는 CrossEntropyLoss 손실 함수와, 가중치를 최적화하는 Adam 옵티마이저를 정의했습니다.
학습 루프 구현: 각 에포크(Epoch)마다 데이터를 배치(Batch) 단위로 나누어 모델에 입력하고, 순전파 → 손실 계산 → 역전파 → 가중치 업데이트의 4단계를 반복하는 코드를 직접 작성했습니다.
4. 모델 평가 및 분석
정확도 측정: 학습이 끝난 후, 학습에 사용하지 않은 테스트 데이터를 사용하여 모델의 **최종 정확도(Accuracy)**를 계산했습니다.
오차 행렬 분석: Scikit-learn 라이브러리를 활용해 모델의 예측값과 실제값을 비교하는 **오차 행렬(Confusion Matrix)**을 만들고 시각화했습니다. 이를 통해 모델이 어떤 클래스를 헷갈려 하는지 한눈에 파악할 수 있었습니다.
5. 성능 최적화
모델 구조 변경: 2계층 모델의 정확도(86.52%)를 개선하기 위해 3계층 모델을 도입하여 정확도를 87.31%로 올렸습니다. 이는 모델의 표현 능력을 높인 결과입니다.
하이퍼파라미터 튜닝: 학습률(Learning Rate)을 **0.01**과 0.0001 등으로 다양하게 변경하며 최적의 값을 찾으려 했습니다. 이 실험을 통해 학습률이 너무 높으면 최적점을 지나치고(오버슈팅), 너무 낮으면 학습이 충분히 진행되지 않는다(언더피팅)는 것을 직접 확인했습니다.

In [ ]:
# 1. 문제 정의 및 데이터 준비
# 문제: Fashion-MNIST 데이터셋을 사용하여 10가지 종류의 의류 이미지를 정확하게 분류하는 이미지 분류(Image Classification) 모델을 만듭니다.
# 데이터: 캐글(Kaggle)에서 다운로드할 수 있는 Fashion-MNIST 데이터셋을 사용합니다. 이 데이터셋은 훈련용 이미지 60,000개와 테스트용 이미지 10,000개로 구성되어 있으며, 각 이미지는 28×28 픽셀의 흑백 이미지입니다.

# 2. 데이터 로딩 및 전처리
# 2.1. 데이터 로드
# 바이너리 파일 형태로 저장되어 gzip, numpy 라이브러리를 사용.
import numpy as np
# import gzip # 이미 압축해제 되어 있어서 필요 없음
# 파일 경로 설정 (여러분의 파일 경로로 수정해주세요)
train_images_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/Fashion MNIST/data/train-images-idx3-ubyte'
train_labels_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/Fashion MNIST/data/train-labels-idx1-ubyte'
test_images_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/Fashion MNIST/data/t10k-images-idx3-ubyte'
test_labels_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/Fashion MNIST/data/t10k-labels-idx1-ubyte'
# offset은 파일의 특정 위치부터 데이터를 읽기 시작하라고 알려주는 기능입니다. Fashion-MNIST 데이터셋의 바이너리 파일은 이미지 데이터나 라벨 데이터가 시작되기 전에, 데이터에 대한 메타정보가 담긴 **헤더(header)**를 가지고 있습니다. 이 헤더 정보를 건너뛰고 순수한 데이터만 읽기 위해 offset을 사용하는 것이죠.
# 이미지 파일 로드 함수 (압축 해제된 파일용)
def load_mnist_images(path:str):
    with open(path, 'rb') as f:
        # 이미지 파일 헤더(16바이트)를 건너뛰고 나머지 데이터를 읽음
        data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
    # 28x28 이미지 크기로 재구성
    return data.reshape(-1, 28, 28)
# 라벨 파일 로드 함수 (압축 해제된 파일용)
def load_mnist_labels(path:str):
    with open(path, 'rb') as f:
        # 라벨 파일 헤더(8바이트)를 건너뛰고 나머지 데이터를 읽음
        return np.frombuffer(f.read(), dtype=np.uint8, offset=8)
# 함수를 사용해 데이터 로드
train_images = load_mnist_images(train_images_path)
train_labels = load_mnist_labels(train_labels_path)
test_images = load_mnist_images(test_images_path)
test_labels = load_mnist_labels(test_labels_path)
# print(f"훈련용 이미지 데이터의 형태: {train_images.shape}")
# print(f"훈련용 라벨 데이터의 형태: {train_labels.shape}")
# print(f"테스트용 이미지 데이터의 형태: {test_images.shape}")
# print(f"테스트용 라벨 데이터의 형태: {test_labels.shape}")

# 3. 데이터 전처리 및 시각화
# 3.1. 이미지 데이터 시각화
import matplotlib.pyplot as plt
# # 3.1.1. train_images에 저장된 첫 번째 이미지 데이터를 선택
# image_to_visualize = train_images[0]
# # 3.1.2. 이미지 시각화를 위해 plt.imshow() 함수를 사용합니다.
# # imshow() 함수는 배열 형태의 데이터를 이미지로 보여주는 역할을 해요.
# # cmap=plt.cm.binary 옵션은 이미지를 흑백으로 나타내라는 의미입니다.
# plt.imshow(image_to_visualize, cmap=plt.cm.binary) # pyright: ignore[reportAttributeAccessIssue]
# # 3.1.3. plt.colorbar() 함수를 사용해 이미지의 픽셀 값 분포를 나타내는 컬러바를 추가합니다.
# # 픽셀 값이 0(검은색)부터 255(흰색)까지 어떻게 분포하는지 한눈에 확인 가능.
# plt.colorbar()
# # 3.1.4. plt.grid(False)를 사용해 이미지 위에 격자가 보이지 않도록 설정합니다.
# #    기본적으로 격자가 나타나는데, 이미지를 깔끔하게 보기 위해 제거하는 거예요.
# plt.grid(False)
# # 3.1.5. plt.show()를 호출해 최종적으로 시각화된 이미지를 화면에 표시합니다.
# plt.show()
# # 3.1.6. 마지막으로, 첫 번째 이미지의 라벨(정답)을 출력합니다.
# #    train_labels는 각 이미지의 정답 배열.
# print(f"첫 번째 이미지의 라벨: {train_labels[0]}")

# 4. 정규화
train_images_normalized = train_images.astype(np.float32) / 255.0
test_images_normalized = test_images.astype(np.float32) / 255.0
# 결과 데이터 형식 확인.
print(train_images_normalized.dtype)  # unint8: unsigned(부호가 없는) int8

# 5. 모델 학습
import torch
import torch.nn as nn
import torch.nn.functional as F
# 5.1. FashionMNIST 모델 클래스 정의
# PyTorch에서는 nn.Module을 상속받아 모델을 클래스 형태로 만듭니다.
# 이렇게 하면 모델의 구조와 동작을 체계적으로 관리할 수 있어요.
class FashionMNISTModel(nn.Module):
    # 5.2. 모델의 각 층(layer) 정의
    # __init__ 함수에서 모델의 구성 요소를 미리 정의합니다.
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        # 5.2.1. Flatten 레이어: 28*28 이미지를 784 픽셀의 1차원 벡터로 변환
        self.flatten = nn.Flatten()
        # 5.2.2. 첫 번째 Dense(Linear) 레이어
        # 784개의 입력을 받아 128개의 뉴런을 가진 층으로 연결.
        # self.fc1 = nn.Linear(28 * 28, 128) # 기존 1층
        ### 9.1.1. 첫 번째 층 뉴런 수 증가
        self.fc1 = nn.Linear(28 * 28, 256) # 기존 1층에서 뉴런 수 증가(256)
        # 5.2.3. 두 번째 Dense(Linear) 레이어
        # 128개의 입력을 받아 10개의 출력 뉴런을 가진 층으로 연결
        # Fashion-MNIST의 클래스가 10개이기 때문
        # self.fc2 = nn.Linear(128, 10) # 기존 2층
        ### 9.1.2. 새로운 중간 층을 추가하고 출력층을 3번째로 이동.
        self.fc2 = nn.Linear(256, 128) # 새로운 중간 층을  추가
        self.fc3 = nn.Linear(128, 10) # 출력층 3층으로 이동.

    # 5.3. 모델의 순전파(foward pass) 정의
    # 이 함수는 입력 데이터가 모델의 층을 통과하는 순서를 정의
    # 데이터가 어떤 과정을 거쳐 최종 결과로 나오는지 결정.
    def forward(self, x):
        # 5.3.1. 입력 이미지를 1차원으로 펼침.
        x = self.flatten(x)
        # 5.3.2. 첫 번째 완전 연결 층을 통과, relu 활성화 함수를 적용
        # ReLu는 활성화 함수이다. 음수 값을 0으로 만들고 양수 값을 그대로 통과시켜 비선형성을 부여.
        x = F.relu(self.fc1(x))
        # 5.3.3. 두 번째 완전 연결 층을 통과시켜 최종 결과를 얻는다.
        # 이 단계의 출력은 각 클래스에 대한 '점수'
        #x = self.fc2(x) # 기존 출력 층
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 출력 층
        # 5.3.4. 최종 결과는 나중에 손실 함수(CrossEntropyLoss)에서 Softmax를 내부적으로 계산
        # Softmax를 직접 적용하지는 않는다.
        return x

# 6. 모델 컴파일
# 6.0. GPU(MPS) 사용 가능 여부 확인 및 장치 설정.
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")
# 6.1. 모델 인스턴스 생성
# 이제 우리가 정의한 FasionMNISTModel 클래스를 사용해 실제 모델 객체를 생성.
# 위에서 설정한 장치(GPU 또는 CPU)로 모델을 보냅니다.
# 이렇게 해야 모델의 가중치가 GPU 메모리에 올라가 GPU 연산을 사용할 수 있습니다.
model = FashionMNISTModel().to(device)
# 6.2. 모델 구조 출력
# print(model)
import torch.optim as optim
# 6.3. 손실 함수 정의
# 모델의 예측과 실제 라벨 간의 오차를 계산하는 함수.
# 다중 클래스 분류 문제에는 CrossEntropyLoss가 가장 널리 사용.
# PyTorch의 이 손실 함수는 내부적으로 Softmax를 포함.
loss_function = nn.CrossEntropyLoss()
# 6.4. 옵티마이저 정의
# 모델의 가중치를 업데이트하여 손실을 최소화하는 역할.
# Adam은 현재 딥러닝에서 가장 많이 사용되는 옵티마이저 중 하나.
# - model.parameters()는 모델이 학습할 모든 가중치와 편향을 넘겨주는 역할
# - lr(learning rate)는 경사 하강법에서 한 번에 움직이는 보폭의 의미
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# 9.2. 하이퍼파라미터 튜닝
# optimizer = optim.Adam(model.parameters(), lr=0.01) # 학습률 크게 증가.
optimizer = optim.Adam(model.parameters(), lr=0.0008) # 학습률 크게 감소.

# 7. 학습루프 생성.
from torch.utils.data import DataLoader, TensorDataset
# 7.1. 데이터셋 및 데이터로더 생성
# 정규화된 numpy 배열 데이터를 PyTorch의 Tensor로 변환.
train_images_tensor = torch.from_numpy(train_images_normalized)
train_labels_tensor = torch.from_numpy(train_labels).long()
# TensorDataset은 이미지와 라벨 텐서를 묶어 데이터셋을 만듭니다.
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
# DataLoader는 학습 시 데이터를 배치(batch) 단위로 불러오는 역할을 합니다.
# shuffle=True는 매 에포크마다 데이터를 무작위로 섞어 모델의 과적합을 방지합니다.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# 7.2. 모델을 장치로 이동
# 모델을 CPU 또는 GPU/MPS로 옮겨줍니다.
model.to(device)
# 7.3. 학습 루프(Training Loop)
#num_epochs = 5
# 9.3. 에포크 수정
num_epochs = 20
print("Training starts...")
# 각 에포크의 시작에서 모델을 학습 모드로 설정합니다.
# 이렇게 하면 드롭아웃 등 학습 시에만 필요한 기능들이 활성화됩니다.
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # 데이터로더에서 이미지와 라벨을 배치 단위로 가져옵니다.
    for images, labels in train_loader:
        # 데이터를 설정한 장치(device)로 보냅니다.
        images = images.to(device)
        labels = labels.to(device)
        # 7.3.1 기울기 초기화
        # 이전 배치 학습에서 계산된 기울기를 0으로 초기화합니다.
        # 이 과정이 없으면 기울기가 계속 누적되어 학습이 이상해집니다.
        optimizer.zero_grad()
        # 7.3.2. 순전파(Foward Pass)
        # 모델에 이미지 데이터를 넣어 예측값을 계산합니다.
        outputs = model(images)
        # 7.3.3. 손실 계산(Loss Calculation)
        # 예측값과 실제 라벨을 비교해 손실(오차)을 계산
        loss = loss_function(outputs, labels)
        # 7.3.4. 역전파(Backward Pass)
        # 손실을 기반으로 각 매개변수에 대한 기울기를 계산
        loss.backward()
        # 7.3.5. 가중치 업데이트(Weight Update)
        # 계산된 기울기를 사용해 모델의 가중치를 업데이트
        optimizer.step()
        # 에포크가 끝날 때마다 현재 손실을 출력합니다.
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        # 에포크가 끝날 때마다 평균 손실을 출력합니다.
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss / len(train_loader):.4f}")
print("Training finished!")

# 8. 모델 평가
# 8.1. 모델을 평가 모드로 설정.
# model.eval()은 학습 시에만 필요한 기능(예: Dropout, Batch Normalization)을 비활성화합니다.
# 이렇게 해야 모델이 일관된 예측 결과를 내놓을 수 있습니다.
model.eval()
# 8.2. 기울기 계산 비활성화
# torch.no_grad() 블록 안에서는 기울기 계산이 이루어지지 않습니다.
# 평가 단계에서는 가중치를 업데이트할 필요가 없으므로, 메모리와 연산 속도를 절약할 수 있습니다.
# 8.2.1. 테스트 데이터셋 및 데이터로더 생성
# 테스트 이미지와 라벨을 PyTorch 텐서로 변환
test_images_tensor = torch.from_numpy(test_images_normalized)
test_labels_tensor = torch.from_numpy(test_labels).long()
# TensorDataset으로 묶고 DataLoader를 생성
# 평가 시에는 데이터 순서를 섞을 필요가 없으므로 suffle=False로 설정.
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
# 8.2.2. 정확도(Accuracy) 계산을 위한 변수 초기화(기울기 계산 비활성화 블록)
correct_predictions = 0
total_samples = 0
# 8.2.3. 테스트 데이터로 순전파 실행.
# 테스트 데이터로더에서 배치 단위로 데이터를 가져옵니다.
for images, labels in test_loader:
    # 데이터와 라벨을 설정한 장치(device)로 보냅니다.
    images = images.to(device)
    labels = labels.to(device)
    # 8.2.3.1. 순전파: 모델에 데이터를 넣어 예측값 얻음.
    outputs = model(images)
    # 8.2.3.2. 예측값 변환
    # outputs은 각 클래스에 대한 점수입니다. 가장 높은 점수를 가진 클래스를 예측값으로 선택합니다.
    # torch.max() 함수는 최댓값과 그 인덱스를 반환합니다. dim=1은 각 행(이미지)에서 최댓값을 찾으라는 의미입니다.
    _, predicted = torch.max(outputs.data, 1)
    # 8.2.3.3. 정확한 예측 수 계산
    # 총 샘플 수를 업데이트
    total_samples += labels.size(0)
    # 예측값과 실제 라벨이 일치하는 개수를 세어 누적
    correct_predictions += (predicted.view(-1) == labels.view(-1)).sum().item()
# 8.2.4. 정확도 계산 및 출력
# (정확히 예측한 샘플 수) / (총 샘플 수)를 계산하여 정확도를 얻습니다.
accuracy = 100 * correct_predictions / total_samples
print(f"Test Accuracy: {accuracy:.2f}%")
print("Evaluation finished!")
# 8.2.5.1. 오차 행렬
from sklearn.metrics import confusion_matrix
import seaborn as sns
model.eval() # 모델을 평가 모드로 설정.
# 8.2.5.2. 예측값과 실제 라벨 수집
# 전체 테스트 데이터셋에 대한 예측값과 실제 라벨을 저장할 리스트를 만듦.
all_predictions = []
all_labels = []
# 기울기 계산을 비활성화하고 평가를 진행.
with torch.no_grad():
    for images, labels in test_loader:
        # 데이터와 라벨을 장치로 보냅니다.
        images, labels = images.to(device), labels.to(device)
        # 순전파: 모델에 이미지를 넣어 예측값을 얻습니다.
        outputs = model(images)
        # 예측값 변환: 가장 높은 점수를 가진 클래스의 인덱스를 예측값으로 선택합니다.
        _, predicted = torch.max(outputs.data, 1)
        # 텐서를 CPU로 옮겨 NumPy 배열로 변환하고 리스트에 추가합니다.
        # Scikit-learn은 PyTorch 텐서 대신 NumPy 배열을 입력으로 받습니다.
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# 8.2.5.3. 오차 행렬 생성
# confusion_matrix 함수를 사용해 예측값과 실제 라벨로 오차 행렬을 만듭니다.
# 오차 행렬은 Numpy 배열 형태로 반환됩니다.
cm = confusion_matrix(all_labels, all_predictions)
print("오차 행렬 (Confusion Matrix):")
print(cm)
# 8.2.5.4. 오차 행렬 시각화
# Seaborn 라이브러리를 사용해 오차 행렬을 히트맵 형태로 시각화합니다.
# fmt='d'는 값을 정수 형태로 표시하라는 의미입니다.
# annot=True는 각 셀에 숫자를 표시하라는 의미입니다.
plt.figure(figsize=(10, 8))
tick_labels = [str(i) for i in range(10)]
sns.heatmap(
    cm, annot=True, fmt="d", cmap="Blues", xticklabels=tick_labels, yticklabels=tick_labels
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# 9. 성능 개선
# 9.1 모델 구조 변경
# 모델에 층을 추가.
# ** 구현을 5.2. 모델 설정에서 수정.
# 9.1.1. 첫 번째 층 뉴런 수 증가
# 9.1.2. 새로운 중간 층을 추가하고 출력층을 3번째로 이동.
# 9.2. 하이퍼파라미터 튜닝(학습률)
# ** 구현을 6.4. 옵티마이저 정의를 수정.

# 결과 보고서
목표: Fashion-MNIST 데이터셋을 사용하여 10가지 종류의 의류 이미지를 정확하게 분류하는 이미지 분류(Image Classification) 모델을 만들어 분류.
데이터: 캐글(Kaggle)에서 다운로드할 수 있는 Fashion-MNIST 데이터셋을 사용합니다. 이 데이터셋은 훈련용 이미지 60,000개와 테스트용 이미지 10,000개로 구성되어 있으며, 각 이미지는 28×28 픽셀의 흑백 이미지입니다.
모델 및 학습: FashionMNISTModel모델을 사용하였다. 레이어는 2 계층. CrossEntropyLoss 손실함수를 사용. Adam 옵티마이저를 사용하여 학습률 (0.01)을 사용. 애포크는 5을 설정하였다. 
결과 및 결론: 최종 모델의 레이어는 3 계층. 뉴런 256개. 학습률 0.008, 에포크 15을 했을 때, 예측률이 88.94%로 가장 높았다. 2계층, 뉴런 128. 학습률 0.01. 애포크 5로 했을 때, 예측률이 87%로 가장 모델 성능과 리소스 사용 대비 성능이 좋았다. 오히려 학습률을 0.0001로 낮추고, 에포크 30으로 했을 때, 예측률이 87%로 떨어졌다. 에포크가 모자란 듯 싶기도 했지만 데이터의 양의 한계 등 이 존재했다고 본다.
무조건 이론 상 학습률이 낮고, 애포크가 높다면 예측률이 높을 수 있지만, 데이터의 한계, 리소스의 한계 등이 존재하여 적절한 타협이 필요하다고 이번 학습을 통해 배울 수 있었다.

# Fashion-MNIST 분류 모델 최종 보고서
1. 프로젝트 목표
본 프로젝트는 Fashion-MNIST 데이터셋을 사용하여 10가지 종류의 의류 이미지를 정확하게 분류하는 딥러닝 모델을 구축하고, 모델의 성능을 최적화하는 것을 목표로 합니다.

2. 사용 데이터셋
데이터셋: Fashion-MNIST
구성: 훈련용 이미지 60,000개, 테스트용 이미지 10,000개
이미지 형태: 28x28 픽셀의 흑백 이미지
전처리: 모든 이미지는 픽셀 값을 255로 나누어 0과 1 사이의 값으로 정규화했습니다.
3. 모델 및 학습 설정
프로젝트 초기에 2개 층의 단순한 모델로 시작하여, 성능 개선을 위해 모델 구조를 변경하고 하이퍼파라미터를 튜닝했습니다.
모델 아키텍처:
최종 모델: DeeperFashionMNISTModel 클래스를 사용했습니다.
구조: 입력층(784) → 은닉층 1 (nn.Linear(784, 256)) → 은닉층 2 (nn.Linear(256, 128)) → 출력층 (nn.Linear(128, 10))
학습 환경:
손실 함수: 다중 클래스 분류에 적합한 nn.CrossEntropyLoss를 사용했습니다.
옵티마이저: 안정적인 성능을 보이는 Adam 옵티마이저를 사용했습니다.
4. 실험 결과 및 분석
다양한 하이퍼파라미터 조합을 시도하며 모델 성능을 평가했습니다.
실험 1: 단순 모델(2계층) + lr=0.01 + Epochs=5
정확도: 약 87.00%
실험 2: 더 깊은 모델(3계층) + lr=0.001 + Epochs=5
정확도: 약 87.31%
분석: 모델의 레이어와 뉴런 수를 늘리자 학습 능력이 향상되어 정확도가 소폭 상승했습니다.
실험 3: 더 깊은 모델(3계층) + lr=0.01 + Epochs=5
정확도: 약 84.67%
분석: 학습률이 너무 높아 모델이 최적의 지점을 지나쳐(overshooting) 성능이 하락했습니다.
실험 4: 더 깊은 모델(3계층) + lr=0.0001 + Epochs=30
정확도: 약 87.00%
분석: 학습률을 낮추자 학습 속도가 느려졌습니다. 에포크를 30으로 늘렸음에도 불구하고, 최적점을 찾기에는 여전히 학습 시간이 부족하여 **언더피팅(underfitting)**이 발생했습니다. 이 결과는 학습률과 에포크 간의 균형이 중요하다는 것을 보여줍니다.
5. 결론 및 향후 계획
본 프로젝트의 실험을 통해 모델 구조와 하이퍼파라미터가 성능에 미치는 영향을 명확히 확인할 수 있었습니다.
최종 모델: 뉴런 256개, 3계층의 모델이 가장 좋은 성능을 보였습니다.
최적의 하이퍼파라미터: 실험 결과에 따르면, 이 모델 구조에서 학습률 **0.001**이 가장 좋은 성능을 냈습니다. 에포크 수를 15회로 늘렸을 때 **88.94%**라는 최고 정확도를 달성하며, 적절한 학습률과 충분한 학습 시간의 중요성을 확인했습니다.
이번 학습을 통해 무작정 이론에 따라 학습률을 낮추고 에포크를 높이는 것이 능사가 아니며, 데이터셋의 규모와 컴퓨팅 리소스 한계를 고려한 적절한 타협점을 찾는 것이 실제 딥러닝 프로젝트에서 매우 중요하다는 것을 배울 수 있었습니다. 향후 더 높은 성능을 위해서는 CNN(합성곱 신경망) 같은 다른 아키텍처를 시도하거나, 데이터 증강 기법을 적용해 볼 수 있을 것입니다.

# 테스트 셀
1. 뉴런 128 * 2계층
- Test Accuracy: 86.52%
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[820   0  27  74   1   1  72   0   5   0]
 [  1 961   1  30   4   0   3   0   0   0]
 [ 14   2 831  24  94   0  35   0   0   0]
 [ 13   2  11 937  23   0  13   0   1   0]
 [  0   0 134  51 778   0  37   0   0   0]
 [  0   0   0   1   0 931   0  44   2  22]
 [137   1 143  65  83   0 563   0   8   0]
 [  0   0   0   0   0  17   0 976   0   7]
 [  7   1   8  10   4   3   7   5 955   0]
 [  0   0   0   0   0   2   1  97   0 900]]
2. 뉴런 수 추가 & 레이어 추가
- Test Accuracy: 87.31%
오차 행렬 (Confusion Matrix):
[[762   3  23  81   1   1 118   0  11   0]
 [  1 970   1  23   3   0   2   0   0   0]
 [ 12   0 836  15  92   0  45   0   0   0]
 [  9   4  14 924  24   0  21   0   4   0]
 [  0   1 119  39 801   0  40   0   0   0]
 [  0   0   0   1   0 954   0  29   1  15]
 [101   0 124  60  90   0 617   0   8   0]
 [  0   0   0   0   0  13   0 920   0  67]
 [  3   0   7   4   4   4   3   4 971   0]
 [  0   0   0   0   0   7   1  16   0 976]]
3. 하이퍼파라미터 튜닝(학습률): 0.001 > 0.01
- Test Accuracy: 84.67%
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[781   4  39  89   2   0  76   0   9   0]
 [  2 954   0  33   4   0   2   0   5   0]
 [  7   1 669  17 204   0  96   0   6   0]
 [ 16   5  11 919  26   0  19   0   4   0]
 [  0   1  49  52 831   0  65   0   2   0]
 [  0   0   0   1   0 907   0  59   8  25]
 [170   2 102  65 114   0 533   1  13   0]
 [  0   0   0   0   0   7   0 965   0  28]
 [  0   0   9   4   5   1  13   8 960   0]
 [  0   0   0   1   0   6   1  44   0 948]]
4. 하이퍼파라미터 튜닝(학습률): 0.001 > 0.0001
- Test Accuracy: 83.82%
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[808   7  12  66   7   0  82   0  17   1]
 [  3 954   7  26   6   0   2   0   2   0]
 [ 17   4 720  13 184   1  53   0   8   0]
 [ 24  17  11 872  40   1  31   0   4   0]
 [  0   3  92  33 806   1  59   0   6   0]
 [  0   0   0   2   0 917   0  50   4  27]
 [146   4 126  57 138   2 500   0  27   0]
 [  0   0   0   0   0  41   0 920   0  39]
 [  2   1   9   8   3   6  13   6 952   0]
 [  0   0   0   2   0  15   0  49   1 933]]
5. 하이퍼파라미터 튜닝(학습률): 0.008, 에포크 10 > 15 으로 증가
- Test Accuracy: 88.94%, 소요시간: 30초
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[879   1  17  21   4   2  74   0   2   0]
 [  1 979   1  12   3   0   4   0   0   0]
 [ 21   1 800  11 112   0  55   0   0   0]
 [ 26  11  12 893  41   0  15   0   2   0]
 [  0   0  73  19 865   0  43   0   0   0]
 [  0   0   0   0   0 977   0   4   1  18]
 [144   1  88  35  83   0 644   0   5   0]
 [  0   0   0   0   0  29   0 926   0  45]
 [  7   0   3   6  10   4   7   3 960   0]
 [  0   0   0   1   0   7   1  20   0 971]]

6. 하이퍼파라미터 튜닝(학습률): 0.008, 에포크 15 > 20 으로 증가
- Test Accuracy: 88.81%, 소요시간: 40.8초
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[727   2   8  17   3   2 236   0   5   0]
 [  2 984   1   7   3   0   2   0   1   0]
 [ 12   1 770  13  91   0 112   0   1   0]
 [  8  10   5 906  28   0  38   0   5   0]
 [  0   1  66  28 816   0  87   0   2   0]
 [  0   0   0   0   0 962   0  29   0   9]
 [ 56   1  54  23  42   0 815   0   9   0]
 [  0   0   0   0   0   5   0 974   0  21]
 [  4   0   1   5   2   6   8   4 970   0]
 [  0   0   0   1   0   6   1  35   0 957]]


7. 하이퍼파라미터 튜닝(학습률): 0.005, 에포크 15 > 30 으로 증가
- Test Accuracy: 88.79%, 소요시간: 1분 1초.
Evaluation finished!
오차 행렬 (Confusion Matrix):
[[780   1  34  22   7   0 148   0   8   0]
 [  1 976   3  12   4   0   4   0   0   0]
 [  8   0 881   8  69   2  32   0   0   0]
 [ 14   4  18 900  39   0  19   0   6   0]
 [  0   0 130  21 821   0  25   0   3   0]
 [  0   0   0   1   0 963   0  19   0  17]
 [ 74   0 146  35  77   0 659   0   9   0]
 [  0   0   0   0   0  10   0 953   0  37]
 [  4   0   6   3   3   3   5   2 974   0]
 [  0   0   0   0   0   8   1  19   0 972]]
